# Rimless Wheel dynamics

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

In [1]:
using LinearAlgebra
using Distributions
using CairoMakie

┌ Info: Precompiling CairoMakie [13f3f980-e62b-5c42-98c6-ff1f3baf88f0]
└ @ Base loading.jl:1664


![](figures/rimlessWheel.svg)

In [76]:
m = 1.0
ℓ = 1.0
γ = 0.5 # 0 < γ < 2π
α = π/6
const g = 9.81

9.81

## Start from stance

In [77]:
function animwheel(θt)

    f = Figure(resolution=(800,700))
    ax = Axis(f[1, 1])

    θk = Observable(0.0)

    # Slope
    poly!(ax, Point2f[(0, 0), (1, 0), (0, tan(γ))])
    CairoMakie.ylims!(ax, [0.,1.])

    # Contact point
    contact = [0.0, tan(γ)]

    xk = @lift([contact[1], contact[1] + ℓ*sin($θk)])
    yk = @lift([contact[2], contact[2] + ℓ*cos($θk)])

    # Standing spoke
    lines!(xk, yk, linewidth=5, color=:black)

    # # Wheel
    centerx = @lift([contact[1]+ℓ/2*sin($θk)])
    centery = @lift([contact[2]+ℓ/2*cos($θk)])
    CairoMakie.scatter!(centerx, centery, marker=:circle, markersize=20, color=:black)

    # Spokes
    for n in 0:5
        xk = @lift([contact[1]+ℓ/2*sin($θk), contact[1]+ℓ/2*sin($θk)+ℓ/2*sin($θk+n*π/3)])
        yk = @lift([contact[2]+ℓ/2*cos($θk), contact[2]+ℓ/2*cos($θk)+ℓ/2*cos($θk+n*π/3)])
        lines!(xk, yk, linewidth=5, color=:black)
    end

    record(f, "rimless-wheel.mp4", θt; framerate = 30) do θ
        θk[] = θ
    end
end

animwheel (generic function with 1 method)

In [96]:
Δt = 0.01
time = range(0, stop=3.0, step=Δt)
T = length(time)

301

In [97]:
function eom(z,t)
    return [z[1] + Δt*z[2]; z[2] + Δt*(g/ℓ*sin(z[1]))]
end

eom (generic function with 1 method)

In [98]:
# Initial condition
z_0 = [0.0, 1.0]

z_ = zeros(2,T)
z_kmin1 = z_0
for (n,t) in enumerate(time)
    z_[:,n] = eom(z_kmin1, t)
    z_kmin1 = z_[:,n]
end

animwheel(z_[1,:])

"rimless-wheel.mp4"